## DATA PROCESING DENUNCIAS v1
 __SECRETARIADO EJECUTIVO __  
 _Reportes de incidencia Enero 2015 - Julio 2018_   
 __ROSA MARTHA PERALTA BLANCO__

#### Descripción

El código calcula el promedio y los limites "normales" para series de tiempo de delitos anuales.  
Los datos se agrupan por año, tipo de delito y estatus metropolitano (Urbe central, urbe extrerior y rural).
A partir de ellos, se obtiene el promedio y los límites, los cuales se obtienen sumando/restando tres desviaciones estandar al promedio (es decir Z-Scores):

- Máximo = mean + (3 * sd)
- Mínimo = mean - (3 * sd)

#### Estructura del input  
- __year__      : Año de denuncia  
- __clv__       : Clave de estado  
- __location_code__ : Clave de municipio  
- __municipio__     : Nombre de municipio  
- __good_legal__    : Buen jurídico afectado  
- __typess__        : Tipo de delito  
- __subtype__       : Subtipo de delito  
- __modality__      : Modalidad del delito  
- __cfinal__        : Clave del delito  
- __tipoz__         : Tipo de municipio: Urbano-Central, Urbano-Exterior, Rural. Delimitación de las Zonas Metropolitanas de México 2010:  
http://Www.Conapo.Gob.Mx/Es/Conapo/Zonas_Metropolitanas_2010  
- __ene-dic__       : Cantidad de denuncias consideradas por mes

#### Notas

Para esta versión despues del script se procesan algunos pasos:  
1. Si limsup y liminf son "NA" se les asigna el valor de mean  
2. En el caso los "NA" de agosto-diciembre de 2018 se remplazan por valores vacios  
3. Calcular total de denuncias de enero a diciembre (a integrar proceso en v2)  
4. _Pendiente_ : Sustituir valores negativos

## CÓDIGO

- __INPUT__

In [1]:
zmetrop <- read.csv("/home/rosa/Documentos/Proyecto_Seguridad/DELITOS HASTA JUNIO2018/EjercicioSeptiembre/base_zmR.csv")
attach(zmetrop)
# CONSTRUYE campo de concatenacion
conca<-paste(year,typess,tipoz)
x<-data.frame(conca,year,clv,location_code,municipio,good_legal,typess,subtype,modality,cfinal,tipoz,ene,feb,mar,abr,may,jun,jul,ago,sep,oct,nov,dic)

2. Cálculo de tabla colapsada por _year_, _typess_ y _tipozm_

In [4]:
#CALCULA EL PROMEDIO
tablemean<-aggregate(x[12:23], by=list(conca), FUN=mean)
#head(tablemean,3)

#CALCULA LA DESVIACIÓN ESTANDAR
tablesd<-aggregate(x[12:23], by=list(conca), FUN=sd)
#head(tablesd,3)

#CALCULA numero de registros
#ASEGURARSE DE QUE LOS REGISTROS SEAN IGUALES
tablelength<-aggregate(x[12:23], by=list(conca), FUN=length)
#head(tablelength,3)

#UNE TABLAS MEAN Y SD
join<-merge(x = tablemean, y = tablesd, by = "Group.1", all = TRUE)
join<-merge(x = join, y = tablelength, by = "Group.1", all = TRUE)

#ORGANIZA EL DataFrame JOIN
join<-data.frame(join$Group.1,join$ene,join$ene.x,join$ene.y,join$feb.x,join$feb.y,join$mar.x,join$mar.y,join$abr.x,join$abr.y,join$may.x,join$may.y,join$jun.x,join$jun.y,join$jul.x,join$jul.y,join$ago.x,join$ago.y,join$sep.x,join$sep.y,join$oct.x,join$oct.y,join$nov.x,join$nov.y,join$dic.x,join$dic.y)
colnames(join)<-c("conca","recurrencia","enemean","enesd","febmean","febsd","marmean","marsd","abrmean","abrsd","maymean","maysd","junmean","junsd","julmean","julsd","agomean","agosd","sepmean","sepsd","octmean","octsd","novmean","novsd","dicmean","dicsd")

head(join,5)

conca,recurrencia,enemean,enesd,febmean,febsd,marmean,marsd,abrmean,abrsd,⋯,agomean,agosd,sepmean,sepsd,octmean,octsd,novmean,novsd,dicmean,dicsd
2015 Aborto Central,87,0.34482759,0.7443990,0.4827586,0.7603865,0.41379310,0.7400774,0.4597701,1.0543884,⋯,0.3448276,0.6255711,0.2068966,0.4855572,0.36781609,0.7940907,0.29885057,0.6308898,0.20689655,0.4350336
2015 Aborto Exterior,12,0.00000000,0.0000000,0.1666667,0.5773503,0.50000000,0.5222330,0.2500000,0.4522670,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.25000000,0.4522670,0.16666667,0.3892495,0.08333333,0.2886751
2015 Aborto Rural,83,0.07228916,0.2605404,0.1084337,0.3128176,0.08433735,0.2795824,0.1084337,0.4420586,⋯,0.2409639,0.5541498,0.1566265,0.4271839,0.07228916,0.3037623,0.07228916,0.2605404,0.16867470,0.5136972
2015 Abuso de confianza Central,206,5.10194175,9.5168296,5.0339806,9.0862432,5.25728155,9.3895022,4.8203883,8.8091843,⋯,5.2864078,9.2425585,5.7087379,9.5500030,5.82038835,10.1386376,5.31067961,9.8590349,5.08252427,8.6801138
2015 Abuso de confianza Exterior,70,0.44285714,0.7349596,0.6285714,1.1315172,0.52857143,1.0864305,0.5428571,0.9276074,⋯,0.6571429,1.1149279,0.5285714,1.0730078,0.68571429,1.3571374,0.58571429,0.9554247,0.38571429,0.7669412


- Prepara la tabla para el loop

In [5]:
#GENERA Y UNE ID
id<-c(1:nrow(join))
join<-data.frame(id,join)

#VALORES DEL FOR
sq<- c(26,24,22,20,18,16,14,12,10,8,6,4)

#GENERA EL DATAFRAME Y MATRIZ INICIAL CON EL ID
concajoin<-data.frame(join$id,join$recurrencia)
limitesjoin<-data.matrix(concajoin)

head(join,5)

id,conca,recurrencia,enemean,enesd,febmean,febsd,marmean,marsd,abrmean,⋯,agomean,agosd,sepmean,sepsd,octmean,octsd,novmean,novsd,dicmean,dicsd
1,2015 Aborto Central,87,0.34482759,0.7443990,0.4827586,0.7603865,0.41379310,0.7400774,0.4597701,⋯,0.3448276,0.6255711,0.2068966,0.4855572,0.36781609,0.7940907,0.29885057,0.6308898,0.20689655,0.4350336
2,2015 Aborto Exterior,12,0.00000000,0.0000000,0.1666667,0.5773503,0.50000000,0.5222330,0.2500000,⋯,0.0000000,0.0000000,0.0000000,0.0000000,0.25000000,0.4522670,0.16666667,0.3892495,0.08333333,0.2886751
3,2015 Aborto Rural,83,0.07228916,0.2605404,0.1084337,0.3128176,0.08433735,0.2795824,0.1084337,⋯,0.2409639,0.5541498,0.1566265,0.4271839,0.07228916,0.3037623,0.07228916,0.2605404,0.16867470,0.5136972
4,2015 Abuso de confianza Central,206,5.10194175,9.5168296,5.0339806,9.0862432,5.25728155,9.3895022,4.8203883,⋯,5.2864078,9.2425585,5.7087379,9.5500030,5.82038835,10.1386376,5.31067961,9.8590349,5.08252427,8.6801138
5,2015 Abuso de confianza Exterior,70,0.44285714,0.7349596,0.6285714,1.1315172,0.52857143,1.0864305,0.5428571,⋯,0.6571429,1.1149279,0.5285714,1.0730078,0.68571429,1.3571374,0.58571429,0.9554247,0.38571429,0.7669412


- loop

In [6]:
for (i in sq)
{
  promedio<-data.frame(join[i])
  sd<-data.frame(join[i+1])
  
  limsup<-promedio + (3*sd)
  colnames(limsup)<-"limsup"
  liminf<-promedio - (3*sd)
  colnames(liminf)<-"liminf"
  
  #promedio<-head(promedio,20)#EJEMPLO DE 20
  #limsup<-head(limsup,20)#EJEMPLO DE 20
  #liminf<-head(liminf,20)#EJEMPLO DE 20
  
  limites<-data.frame(promedio,limsup,liminf)
  limitesmatrix<-data.matrix(limites)
  limitesjoin<-cbind(limitesmatrix,limitesjoin)
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
}

head(limitesjoin,5)

[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"
[1] "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~READY~~~~~~~

enemean,limsup,liminf,febmean,limsup,liminf,marmean,limsup,liminf,abrmean,⋯,limsup,liminf,novmean,limsup,liminf,dicmean,limsup,liminf,join.id,join.recurrencia
0.34482759,2.5780247,-1.8883695,0.4827586,2.763918,-1.798401,0.41379310,2.6340252,-1.8064390,0.4597701,⋯,2.750088,-2.0144560,0.29885057,2.1915199,-1.5938188,0.20689655,1.5119973,-1.0982042,1,87
0.00000000,0.0000000,0.0000000,0.1666667,1.898717,-1.565384,0.50000000,2.0666989,-1.0666989,0.2500000,⋯,1.606801,-1.1068011,0.16666667,1.3344151,-1.0010817,0.08333333,0.9493587,-0.7826921,2,12
0.07228916,0.8539102,-0.7093319,0.1084337,1.046886,-0.830019,0.08433735,0.9230846,-0.7544099,0.1084337,⋯,0.983576,-0.8389977,0.07228916,0.8539102,-0.7093319,0.16867470,1.7097663,-1.3724169,3,83
5.10194175,33.6524304,-23.4485470,5.0339806,32.292710,-22.224749,5.25728155,33.4257882,-22.9112251,4.8203883,⋯,36.236301,-24.5955245,5.31067961,34.8877842,-24.2664250,5.08252427,31.1228658,-20.9578172,4,206
0.44285714,2.6477360,-1.7620217,0.6285714,4.023123,-2.765980,0.52857143,3.7878630,-2.7307201,0.5428571,⋯,4.757127,-3.3856979,0.58571429,3.4519885,-2.2805599,0.38571429,2.6865378,-1.9151092,5,70


5. Une a base inicial y prepara el output

In [9]:
#union de datos de limites con la base de todos los registros
bdura<-as.data.frame(limitesjoin)
#names(bdura)
colnames(bdura)<-c("enemean","enelimsup","eneliminf","febmean","feblimsup","febliminf","marmean","marlimsup","marliminf","abrmean","abrlimsup","abrliminf","maymean","maylimsup","mayliminf","junmean","junlimsup","junliminf","julmean","jullimsup","julliminf","agomean","agolimsup","agoliminf","sepmean","seplimsup","sepliminf","octmean","octlimsup","octliminf","novmean","novlimsup","novliminf","dicmean","diclimsup","dicliminf","id","recurrencia")
join2<-data.frame(join$id,join$conca)
colnames(join2)<-c("id","conca")
jrs<-merge(x =join2, y = bdura, by = "id", all = TRUE)
jrs<-merge(x =x, y = jrs, by = "conca", all = TRUE)

jrs$conca<-NULL

#Reemplaza valores nulos "NA"
jrs[is.na(jrs)] <- 9999999

head(jrs,5)

year,clv,location_code,municipio,good_legal,typess,subtype,modality,cfinal,tipoz,⋯,octmean,octlimsup,octliminf,novmean,novlimsup,novliminf,dicmean,diclimsup,dicliminf,recurrencia
2015,9,9017,Venustiano Carranza,La vida y la integridad corporal,Aborto,Aborto,Aborto,VICaboOOOOOO,Central,⋯,0.3678161,2.750088,-2.014456,0.2988506,2.19152,-1.593819,0.2068966,1.511997,-1.098204,87
2015,13,13077,Tulancingo de Bravo,La vida y la integridad corporal,Aborto,Aborto,Aborto,VICaboOOOOOO,Central,⋯,0.3678161,2.750088,-2.014456,0.2988506,2.19152,-1.593819,0.2068966,1.511997,-1.098204,87
2015,9,9013,Xochimilco,La vida y la integridad corporal,Aborto,Aborto,Aborto,VICaboOOOOOO,Central,⋯,0.3678161,2.750088,-2.014456,0.2988506,2.19152,-1.593819,0.2068966,1.511997,-1.098204,87
2015,14,14120,Zapopan,La vida y la integridad corporal,Aborto,Aborto,Aborto,VICaboOOOOOO,Central,⋯,0.3678161,2.750088,-2.014456,0.2988506,2.19152,-1.593819,0.2068966,1.511997,-1.098204,87
2015,21,21114,Puebla,La vida y la integridad corporal,Aborto,Aborto,Aborto,VICaboOOOOOO,Central,⋯,0.3678161,2.750088,-2.014456,0.2988506,2.19152,-1.593819,0.2068966,1.511997,-1.098204,87


6. __Output__

In [10]:
write.csv(jrs, file = "/home/rosa/Documentos/Proyecto_Seguridad/DELITOS HASTA JUNIO2018/EjercicioSeptiembre/jrs_final9k.csv", row.names = TRUE)